# Project 

## Project Description



## Part 1: Data Retrieval & Preprocessing


In [ ]:
# Import Libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emoji

import nltk
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer


from datasets import load_dataset
import transformers
import simpletransformers
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModel,
)
from sklearn.metrics import classification_report


### once you downloaded vader successfully you do not need to do this again.
### You can command it out in your personal copy as I did below to skip this.

# nltk.download('vader_lexicon', quiet=False)

# Define file paths
sentiment_train_file = "Datasets/Sentiment_Analysis/sentiment_analysis.csv"         
sentiment_test_file = "Datasets/Test Sets/sentiment-topic-test.tsv"

### Part 1.2 Sentiment Analysis Preprocessing

In [ ]:
# Load the dataset
sentiment_train_df = pd.read_csv(sentiment_train_file)

# Rename the second column entries
sentiment_train_df.iloc[:, 1] = sentiment_train_df.iloc[:, 1].replace({
    'positive': 'POS',
    'neutral': 'NEU',
    'negative': 'NEG'
})
# Remove rows where the 'text' column has more than 128 characters
sentiment_train_df = sentiment_train_df[sentiment_train_df.iloc[:, 0].str.len() <= 128]

# Save the updated training data back to the file
sentiment_train_df.to_csv(sentiment_train_file, index=False)


(467, 2)

In [142]:
# Load the sentiment test file into a DataFrame
sentiment_test_df = pd.read_csv(sentiment_test_file, sep='\t')



# Rename and reorganize the columns
sentiment_test_df = sentiment_test_df.rename(columns={
    'sentence_id': 'ID',
    'topic': 'Relevant Topic',
    'sentiment': 'Sentiment',
    'sentence': 'Text'
})[['Text', 'Sentiment', 'Relevant Topic', 'ID']]

# Rename the third column entries
sentiment_test_df.iloc[:, 1] = sentiment_test_df.iloc[:, 1].replace({
    'positive': 'POS',
    'neutral': 'NEU',
    'negative': 'NEG'
})


## Part 2 Processing & Modeling

### Part 2.1: Sentiment Analysis using transformers

In [121]:
# Load the model
sentiment_analysis = pipeline("sentiment-analysis", 
                            model="finiteautomata/bertweet-base-sentiment-analysis", 
                            tokenizer="finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
# Initialize lists to store predicted sentiments and true sentiments
predicted_sentiments = []
true_sentiments = []

# Iterate over each row in the training set
for _, row in sentiment_test_df.iterrows():
    # Perform sentiment analysis on the text
    result = sentiment_analysis(row['Text'])[0]
    predicted_sentiments.append(result['label'])
    
    # Save the true sentiment
    true_sentiments.append(row['Sentiment'])

In [123]:
# Generate the classification report
Sentiment_Analysis_Report = classification_report(true_sentiments, predicted_sentiments)
print(Sentiment_Analysis_Report)

              precision    recall  f1-score   support

         NEG       0.83      0.83      0.83         6
         NEU       0.80      0.67      0.73         6
         POS       0.86      1.00      0.92         6

    accuracy                           0.83        18
   macro avg       0.83      0.83      0.83        18
weighted avg       0.83      0.83      0.83        18

